In [ ]:
# # access.log
# 2026-02-07T10:00:01Z auth 200 120
# 2026-02-07T10:00:02Z auth 503 80
# 2026-02-07T10:00:03Z users 200 200

# # comment
# 2026-02-07T10:00:04Z users 500 220
# 2026-02-07T10:00:05Z users 404 90
# 2026-02-07T10:00:06Z orders 502 300
# 2026-02-07T10:00:07Z orders 200 310
# 2026-02-07T10:00:08Z orders 200 290
# 2026-02-07T10:00:09Z auth 200 110
# 2026-02-07T10:00:10Z auth 500 130
# 2026-02-07T10:00:11Z users 200 210
# 2026-02-07T10:00:12Z users 503 230

# errors_total=6
# top_slowest:
# - orders avg_ms=300.0
# - users avg_ms=190.0


import sys

def analyze(lines):
    totals = {}   # service -> total latency
    counts = {}   # service -> count
    errors = 0

    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"): #changed to or
            continue

        parts = line.split() 
        #--> ["ts","users","500","200"] --> list of "str"

        svc = parts[1] 
        # svc = "users"-->str

        status = int(parts[2])  
        # status = "500"

        ms = int(parts[3])
        #ms = 220

        totals[svc] = totals.get(svc,0) + ms
        #{"users":220}

        counts[svc] = counts.get(svc, 0) + 1
        #{"users":1}

        if status >= 500:
            errors += 1
            #error = 1

    avgs = {}
    for svc in totals:
        avgs[svc] = totals[svc] / counts[svc]
        #avgs = {"users":}   -->200/1--> {"users":200}
        #  

    top2 = sorted(avgs, key=avgs.keys(), reverse=True)[:2]
    return errors, top2, avgs

def main(argv):
    if len(argv) != 1:
        raise ValueError("usage: slow.py <logfile>")

    with open(argv[0]) as f:
        errors, top2, avgs = analyze(f)

    print("errors_total=" + str(errors))
    print("top_slowest:")
    for svc in top2:
        print("- " + svc + " avg_ms=" + str(avgs[svc]))

if __name__ == "__main__":
    main(sys.argv[1:])